# Pedro - Short Queeze Predictor
---

### Imports

In [29]:
# Import necessary libraries for the project
import os
import numpy as np
import pandas as pd
import yfinance as yf
from glob import glob
from pathlib import Path
from dotenv import load_dotenv
import alpaca_trade_api as tradeapi
import pandas_market_calendars as mcal
from sklearn.preprocessing import StandardScaler,OneHotEncoder
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras.layers import Dense
from tensorflow.keras.models import Sequential

### Short Interest Data Collection

In [30]:
# Load and preprocess short interest data
short_df = pd.read_csv(Path("Resources/ShortFloat.csv"))
short_df.rename(columns={'ShortSqueeze.com Short Interest Data': 'Company Name'}, inplace=True)

# Dropping irrelevant columns
columns_to_drop = [
    'Total Short Interest', 'Days to Cover', 'Performance (52-wk)', 'Short: Prior Mo', '% Change Mo/Mo',
    'Shares: Float', 'Avg. Daily Vol.', 'Shares: Outstanding', 'Short Squeeze Ranking™', '% from 52-wk High',
    '(abs)', '% from 200 day MA', '(abs).1', '% from 50 day MA', '(abs).2', '% Insider Ownership',
    '% Institutional Ownership'
]
columns_to_drop = [col for col in columns_to_drop if col in short_df.columns]
short_df.drop(columns_to_drop, axis=1, inplace=True)

# Convert 'Short % of Float' and 'Market Cap' to numeric and apply filters
short_df['Short % of Float'] = pd.to_numeric(short_df['Short % of Float'], errors='coerce')
short_df = short_df[short_df['Short % of Float'] >= 17]
short_df['Market Cap'] = pd.to_numeric(short_df['Market Cap'], errors='coerce')
short_df = short_df[short_df['Market Cap'] >= 300000000]

/Users/regulolanz/anaconda3/envs/dev/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3457: DtypeWarning: Columns (2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,24) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


### Date Cleaning and Mapping

In [31]:
# Convert 'Record Date' column to datetime, sort the dataframe by 'Record Date'.
date_mapping = {
    'JanA': '01-11', 'JanB': '01-25',
    'FebA': '02-09', 'FebB': '02-27',
    'MarA': '03-09', 'MarB': '03-24',
    'AprA': '04-12', 'AprB': '04-25',
    'MayA': '05-09', 'MayB': '05-24',
    'JunA': '06-09', 'JunB': '06-27',
    'JulA': '07-12', 'JulB': '07-25',
    'AugA': '08-09', 'AugB': '08-24',
    'SepA': '09-12', 'SepB': '09-26',
    'OctA': '10-10', 'OctB': '10-24',
    'NovA': '11-09', 'NovB': '11-27',
    'DecA': '12-11', 'DecB': '12-27',
}
short_df['Record Date'] = pd.to_datetime(short_df['Record Date'].str.replace(r'(\d{4})-(\w+)', lambda m: f'{m.group(1)}-{date_mapping[m.group(2)]}'))
short_df.sort_values('Record Date', inplace=True)
short_df.reset_index(drop=True, inplace=True)

/Users/regulolanz/anaconda3/envs/dev/lib/python3.7/site-packages/ipykernel_launcher.py:16: FutureWarning: The default value of regex will change from True to False in a future version.
  


### Insider Trading Data Collection

In [32]:
# Load insider trading data, remove dollar symbols, and convert columns to numeric
insider_df = pd.read_csv("Resources/InsiderTrading.csv")
insider_df['Total Amount'] = insider_df['Total Amount'].replace({'\$': '', ',': ''}, regex=True).astype(float)
insider_df['Share Price'] = insider_df['Share Price'].replace({'\$': '', ',': ''}, regex=True).astype(float)
insider_df['Date'] = pd.to_datetime(insider_df['Date'])
insider_df['Total Amount'] = pd.to_numeric(insider_df['Total Amount'], errors='coerce')
insider_df = insider_df[insider_df['Total Amount'] >= 1000000]

### Data Merge

In [33]:
# Merge short_df and insider_df based on the Symbol column, rename columns, and select necessary columns
merged_df = pd.merge(short_df, insider_df, on='Symbol')
merged_df['Share Price'] = merged_df['Share Price'].replace({'\$': '', ',': ''}, regex=True).astype(float)
merged_df = merged_df[['Symbol', 'Short % of Float', 'Total Amount', 'Record Date', 'Share Price', 'Company Name', 'Sector', 'Industry', 'Date']]

### Data Cleaning and Filtering

In [34]:
# Calculate the difference between 'Date' and 'Record Date' for each row
merged_df['Date_diff'] = (merged_df['Date'] - merged_df['Record Date']).dt.days

# Filter out rows where 'Date_diff' is more than 30 and drop unnecessary columns
merged_df = merged_df[merged_df['Date_diff'] >= 0]
merged_df.sort_values(['Symbol', 'Date_diff'], inplace=True)
merged_df.drop_duplicates(subset=['Symbol', 'Date'], keep='first', inplace=True)
merged_df = merged_df[merged_df['Date_diff'] <= 30]
merged_df.drop(columns=['Record Date', 'Date_diff'], inplace=True)

# Reorder columns
new_column_order = ['Symbol', 'Short % of Float', 'Total Amount', 'Date', 'Company Name', 'Sector', 'Industry']
merged_df = merged_df[new_column_order]

In [35]:
merged_df

,Symbol,Short % of Float,Total Amount,Date,Company Name,Sector,Industry
32,AKRO,26.46,10400000.00,2022-09-19,Akero Therapeutics Inc,Healthcare,Biotechnology
650,ASAN,26.61,3498910.93,2022-02-09,Asana Inc. Class A Common Stock,Technology,Software - Application
641,ASAN,26.61,1400743.32,2022-02-10,Asana Inc. Class A Common Stock,Technology,Software - Application
637,ASAN,26.61,1035540.04,2022-02-11,Asana Inc. Class A Common Stock,Technology,Software - Application
632,ASAN,26.61,12355033.74,2022-02-16,Asana Inc. Class A Common Stock,Technology,Software - Application
...,...,...,...,...,...,...,...
5173,TCDA,19.15,1717048.89,2022-09-06,Tricida Inc,Healthcare,Drug Manufacturers - Specialty & Generic
5172,TCDA,19.15,5397432.60,2022-09-07,Tricida Inc,Healthcare,Drug Manufacturers - Specialty & Generic
4401,UWMC,24.79,1010056.36,2022-05-12,UWM Holdings Corporation Class A Common Stock,Financial Services,Mortgage Finance
415,W,21.38,1222400.00,2022-03-03,Wayfair Inc Class A,Consumer Cyclical,Internet Retail


### Data Augmentation

In [ ]:
# Create new columns for Close Prices at future dates and calculate Returns.
nyse = mcal.get_calendar('NYSE')
desired_days = [1, 2, 3, 4, 5, 7]
for day in desired_days:
    merged_df[f'Close Price Day {day}'] = np.nan

for idx, row in merged_df.iterrows():
    trading_days = nyse.valid_days(start_date=row['Date'], end_date=row['Date'] + pd.DateOffset(days=10))

    for day in desired_days:
        if day <= len(trading_days):
            data = yf.download(row['Symbol'], start=trading_days[day - 1], end=trading_days[day - 1] + pd.DateOffset(days=1))
            if not data.empty:  
                merged_df.loc[idx, f'Close Price Day {day}'] = data['Close'][0] 

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%********

### Calculate Returns and Highest Day Return

In [ ]:
for day in [5, 7]: 
    merged_df[f'Return ({day} Days)'] = ((merged_df[f'Close Price Day {day}'] - merged_df['Close Price Day 1']) / merged_df['Close Price Day 1']) * 100

merged_df['Highest Day Return'] = merged_df[[f'Return ({day} Days)' for day in [5, 7]]].max(axis=1)
merged_df['Highest Close Price'] = merged_df[[f'Close Price Day {day}' for day in desired_days]].max(axis=1)

# Format the 'Close Price' and 'Return' columns to have 2 decimal places
for col in merged_df.columns:
    if 'Close Price' in col or 'Return' in col:
        merged_df[col] = merged_df[col].round(2)

# Drop NaNs and reset the index
merged_df.dropna(inplace=True)
merged_df.reset_index(drop=True, inplace=True)

merged_df

In [ ]:
merged_df.head()

In [ ]:
merged_df['Short Squeeze'] = 0
merged_df.loc[merged_df['Highest Day Return'] >= 20, 'Short Squeeze'] = 1
merged_df.loc[merged_df['Highest Day Return'] < 20, 'Short Squeeze'] = 0

merged_df

In [ ]:
# Transpose the dataframe for further analysis
transposed_df = merged_df.T
transposed_df = transposed_df.drop(['Company Name', 'Date', 'Sector', 'Industry'])
transposed_df.set_axis(transposed_df.loc['Symbol'], axis=1, inplace=True)
transposed_df.drop('Symbol', inplace=True)

# Display the transposed dataframe
transposed_df.head()

## Data Preparation

In [ ]:
# Review the data types associated with the columns
merged_df.dtypes

In [ ]:
# Create a list of categorical variables 
categorical_variables = list(merged_df.select_dtypes("object").columns)

# Display the categorical variables list
display(categorical_variables)

In [ ]:
enc = OneHotEncoder(sparse=False)

In [ ]:
# Encode the categorical variables using OneHotEncoder
encoded_data = enc.fit_transform(merged_df[categorical_variables])

In [ ]:
# Create a DataFrame with the encoded variables
encoded_df = pd.DataFrame(encoded_data, columns = enc.get_feature_names(categorical_variables))

# Review the DataFrame
encoded_df.head()

In [ ]:
# Add the numerical variables from the original DataFrame to the one-hot encoding DataFrame
encoded_df = pd.concat([merged_df.select_dtypes(["int64", "float64"]), encoded_df], axis=1)

# Review the DataFrame
encoded_df.head()

In [ ]:
# Segment the features from the target
X = encoded_df.drop(columns='Short Squeeze')
y = encoded_df['Short Squeeze']

In [ ]:
# Split data
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)

In [ ]:
# Create a StandardScaler instance
scaler = StandardScaler()

# Fit the scaler to the features training dataset
X_scaler = scaler.fit(X_train)

# Fit the scaler to the features training dataset
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)